In [0]:
import pandas as pd
import os
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import LeakyReLU,PReLU
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split


In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
base_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Datasets/all_data_imputed_v2_pca.csv')

In [0]:
X = base_data.iloc[:, :-1]
y = base_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=3)

In [0]:
def generate_model(dropout, neuronPct, neuronShrink):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    
    # Construct neural network
    # kernel_initializer = tensorflow.keras.initializers.he_uniform(seed=None)
    model = Sequential()

    # So long as there would have been at least 25 neurons and fewer than 10
    # layers, create a new layer.
    layer = 0
    while neuronCount>25 and layer<10:
        # The first (0th) layer needs an input input_dim(neuronCount)
        if layer==0:
            model.add(Dense(neuronCount, input_dim=X.shape[1],
                            activation="relu"))
        else:
            model.add(Dense(neuronCount, activation="relu")) 
        layer += 1

        # Add dropout after each hidden layer
        model.add(Dropout(dropout))

        # Shrink neuron count for each layer
        neuronCount = neuronCount * neuronShrink

    model.add(Dense(1,activation='sigmoid')) # Output
    return model

In [10]:

# Generate a model and see what the resulting structure looks like.
model = generate_model(dropout=0.2, neuronPct=0.1, neuronShrink=0.25)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               13000     
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 125)               62625     
_________________________________________________________________
dropout_1 (Dropout)          (None, 125)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 31)                3906      
_________________________________________________________________
dropout_2 (Dropout)          (None, 31)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3

In [26]:
def evaluate_network(dropout,lr,neuronPct,neuronShrink):
    SPLITS = 2
    # Splits tells us how many times it will split the data randomly and create the model
    
    # Bootstrap
    boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)

     # Track progress
    mean_benchmark = []
    epochs_needed = []
    num = 0

    # Loop through samples
    for train, test in boot.split(X, y):
        start_time = time.time()
        num+=1

        # Split train and test
        x_train = X.iloc[train]
        y_train = y.iloc[train]
        x_test = X.loc[test]
        y_test = y.iloc[test]

        model = generate_model(dropout, neuronPct, neuronShrink)
        model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr))
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=100, verbose=0, mode='auto', restore_best_weights=True)

        # Train on the bootstrap sample
        model.fit(x_train,y_train,validation_data=(x_test,y_test),
                  callbacks=[monitor],verbose=0,epochs=5)
        
        epochs = monitor.stopped_epoch
        epochs_needed.append(epochs)

        # Predict on the out of boot (validation)
        pred = model.predict(x_test)

        # Measure this bootstrap's log loss
        # y_compare = np.argmax(y_test,axis=1) # For log loss calculation
        score = metrics.log_loss(y_test, pred, eps= 0.0001)
        mean_benchmark.append(score)
        m1 = statistics.mean(mean_benchmark)
        m2 = statistics.mean(epochs_needed)
        mdev = statistics.pstdev(mean_benchmark)

        # Record this iteration
        time_took = time.time() - start_time
        
    tensorflow.keras.backend.clear_session()
    return (-m1)

print(evaluate_network(
    dropout=0.2,
    lr=1e-3,
    neuronPct=0.2,
    neuronShrink=0.2))

-0.4268185777018235


In [22]:
pip install bayesian-optimization

In [25]:
from bayes_opt import BayesianOptimization
import time

# Supress NaN warnings
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

# Bounded region of parameter space
pbounds = {'dropout': (0.0, 0.499),
           'lr': (0.0, 0.1),
           'neuronPct': (0.01, 1),
           'neuronShrink': (0.01, 1)
          }

pboundsTest = {'dropout': (0.0, 0.15),
           'lr': (0.05, 0.1),
           'neuronPct': (0.01, 0.2),
           'neuronShrink': (0.01, 0.2)
          }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pboundsTest,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1,
)

start_time = time.time()
optimizer.maximize(init_points=10, n_iter=100,)
time_took = time.time() - start_time

print(f"Total runtime: {hms_string(time_took)}")
print(optimizer.max)

|   iter    |  target   |  dropout  |    lr     | neuronPct | neuron... |
-------------------------------------------------------------------------
|  1        | -0.5486   |  0.06255  |  0.08602  |  0.01002  |  0.06744  |
|  2        | -0.4854   |  0.02201  |  0.05462  |  0.04539  |  0.07566  |
|  3        | -0.5486   |  0.05952  |  0.07694  |  0.08965  |  0.1402   |
|  4        | -0.5484   |  0.03067  |  0.09391  |  0.0152   |  0.1374   |
|  5        | -0.5507   |  0.0626   |  0.07793  |  0.03667  |  0.04764  |
|  6        | -0.5496   |  0.1201   |  0.09841  |  0.06955  |  0.1415   |
|  7        | -0.5528   |  0.1315   |  0.09473  |  0.02616  |  0.01742  |
|  8        | -0.5497   |  0.02547  |  0.09391  |  0.02869  |  0.09001  |
|  9        | -0.5504   |  0.1437   |  0.07666  |  0.1415   |  0.06995  |
|  10       | -0.5483   |  0.103    |  0.09173  |  0.01347  |  0.1525   |
|  11       | -0.548    |  0.15     |  0.05     |  0.2      |  0.2      |
|  12       | -0.4264   |  0.0      | 